In [12]:
import pandas as pd


# Import CSV of fitbit weartime using heartrates
df = pd.read_csv('fitbitWearTimeViaHR_merged.csv')

Handle the weartime data: return Ids, average wear time via heartrate data, number of days worn, and largest strand of consecutive days worn.

In [14]:
df['Day'] = pd.to_datetime(df['Day'], format='%m/%d/%Y', errors='coerce')
df

,Id,Day,TotalMinutesWearTime,PercentageWearTime
0,10001,2024-10-03,0,0.000000
1,10001,2024-10-04,0,0.000000
2,10001,2024-10-05,0,0.000000
3,10001,2024-10-06,0,0.000000
4,10001,2024-10-07,0,0.000000
...,...,...,...,...
4947,56214,2024-10-06,0,0.000000
4948,56214,2024-10-07,0,0.000000
4949,56214,2024-10-08,1,0.069444
4950,56214,2024-10-09,72,5.000000


In [35]:
# Return wear time and sync data stats!

# FIND CONSECUTIVE DAYS
def consecutive_days(group, minsGroup = None):
    conDays = 0
    currentStreak = 0
    previousDay = None

    for i, dateTime in enumerate(group):
        currentDay = dateTime
        if(previousDay is None or abs((currentDay - previousDay).days) == 1):
            if(minsGroup is not None):
                if(minsGroup.iloc[i] > 0):
                    currentStreak += 1
                else:
                    currentStreak = 0
            else:
                currentStreak += 1
            if(currentStreak > conDays):
                conDays = currentStreak
        else :
            if(abs((currentDay - previousDay).days) != 0):
                currentStreak = 1
        previousDay = currentDay
    return conDays
    

# We will keep track of average documented weartime, total number of days worn, number of syncs, and last time synced.
reminder_wearDf = df.groupby('Id').apply(lambda x: pd.Series({
    'AverageWearTime': x['PercentageWearTime'].mean(),
    'DaysWorn': (x['TotalMinutesWearTime'] > 0).sum(),
    'ConsecutiveDaysWorn': consecutive_days(x['Day'], x['TotalMinutesWearTime'])
})).reset_index()
reminder_wearDf

,Id,AverageWearTime,DaysWorn,ConsecutiveDaysWorn
0,10001,0.000000,0.0,0.0
1,10101,0.000000,0.0,0.0
2,10102,0.000000,0.0,0.0
3,10103,0.000000,0.0,0.0
4,10104,0.000000,0.0,0.0
...,...,...,...,...
614,56210,0.000000,0.0,0.0
615,56211,85.685764,8.0,8.0
616,56212,0.000000,0.0,0.0
617,56213,68.611111,7.0,5.0


Handle the sync data: return Ids, last day synced, number of days synced, and largest strand of consecutive days synced.

In [36]:
# Read sync data
syncDf = pd.read_csv('syncEvents_merged.csv')
syncDf['DateTime'] = pd.to_datetime(syncDf['DateTime'], format='%m/%d/%Y %I:%M:%S %p', errors='coerce').dt.date


syncDf

,Id,DateTime,SyncDateUTC,Provider,DeviceName
0,51102,2024-10-03,10/3/2024 5:06:10 AM,Fitbit,Inspire 3
1,51102,2024-10-03,10/3/2024 5:21:16 AM,Fitbit,Inspire 3
2,51102,2024-10-03,10/3/2024 5:36:21 AM,Fitbit,Inspire 3
3,51102,2024-10-03,10/3/2024 5:51:28 AM,Fitbit,Inspire 3
4,51102,2024-10-03,10/3/2024 6:06:34 AM,Fitbit,Inspire 3
...,...,...,...,...,...
35450,56213,2024-10-09,10/9/2024 6:09:16 PM,Fitbit,Inspire 3
35451,56214,2024-10-09,10/9/2024 2:03:58 PM,Fitbit,Inspire 3
35452,56214,2024-10-09,10/9/2024 10:15:51 PM,Fitbit,Inspire 3
35453,56214,2024-10-10,10/10/2024 12:09:37 PM,Fitbit,Inspire 3


In [37]:
# Function to determine days synced
def uniqueDays(group):
    days = 0
    previousDay = None

    for dateTime in group:
        currentDay = str(dateTime)
        if(previousDay is None or currentDay[8:10] != previousDay[8:10]):
            days += 1
        previousDay = currentDay
    return days


# Sort last day synced and total number of syncs
reminder_syncDf = syncDf.groupby('Id').agg(
    LastDaySynced = ('DateTime', 'last'),
    DaysSynced = ('DateTime', uniqueDays),
    ConsecutiveDaysSynced = ('DateTime', consecutive_days)
).reset_index()

reminder_syncDf

,Id,LastDaySynced,DaysSynced,ConsecutiveDaysSynced
0,51102,2024-10-10,6,4
1,51103,2024-10-10,8,8
2,51105,2024-10-07,5,5
3,51106,2024-10-10,5,3
4,51110,2024-10-03,1,1
...,...,...,...,...
98,56111,2024-10-10,8,8
99,56112,2024-10-10,3,3
100,56211,2024-10-10,4,2
101,56213,2024-10-09,4,4


Combine weartime and sync dataframes! (NaN may appear where data is missing.)

In [41]:
# Combine Weartime and Sync dataframes
reminder_df = pd.merge(reminder_wearDf, reminder_syncDf, on = 'Id', how = 'outer')
reminder_df.fillna(0, inplace=True)
reminder_df

,Id,AverageWearTime,DaysWorn,ConsecutiveDaysWorn,LastDaySynced,DaysSynced,ConsecutiveDaysSynced
0,10001,0.000000,0.0,0.0,0,0.0,0.0
1,10101,0.000000,0.0,0.0,0,0.0,0.0
2,10102,0.000000,0.0,0.0,0,0.0,0.0
3,10103,0.000000,0.0,0.0,0,0.0,0.0
4,10104,0.000000,0.0,0.0,0,0.0,0.0
...,...,...,...,...,...,...,...
614,56210,0.000000,0.0,0.0,0,0.0,0.0
615,56211,85.685764,8.0,8.0,2024-10-10,4.0,2.0
616,56212,0.000000,0.0,0.0,0,0.0,0.0
617,56213,68.611111,7.0,5.0,2024-10-09,4.0,4.0


Ignore personalized messages for now:

In [39]:
# Determine what messages people should receive?
# Set threshold values for consistency
# min_consecutive_days_worn = 3
# min_average_wear_time = 50.0
# max_days_not_synced = 3

# # Generate personalized reminders
# reminder_df['Reminder'] = ''

# for index, row in reminder_df.iterrows():
#     reminder = ''
    
#     if row['ConsecutiveDaysWorn'] < min_consecutive_days_worn:
#         reminder += f"You have only worn the device for {row['ConsecutiveDaysWorn']} consecutive days in the last month. Please aim to wear it for at least {min_consecutive_days_worn} consecutive days! "
    
#     if row['AverageWearTime'] < min_average_wear_time:
#         reminder += f"In the last month, your average wear time was {row['AverageWearTime']:.2f}%. Next month, please remember to wear it more consistently to further help our reseach. Let's aim for {min_consecutive_days_worn} consecutive days!"
    
#     last_sync_date = pd.to_datetime(row['LastDaySynced'])
#     days_since_last_sync = (pd.Timestamp.now() - last_sync_date).days
    
#     if days_since_last_sync > max_days_not_synced:
#         reminder += f"It has been {days_since_last_sync} days since your last sync. Please remember to sync your device regularly. "
    
#     if row['ConsecutiveDaysSynced'] < min_consecutive_days_worn:
#         reminder += f"You have only synced the device for {row['ConsecutiveDaysSynced']} consecutive days. Please aim to sync it for at least {min_consecutive_days_worn} consecutive days. "
    
#     if reminder == '':
#         reminder = "Great job! You have been wearing and syncing your device consistently. Keep up the good work!"
    
#     reminder_df.at[index, 'Reminder'] = reminder.strip()

# reminder_df

Return a new CSV file, reminder_ids.csv, with all of the data!

In [42]:
# Save the reminders to a CSV file
reminder_df.to_csv('reminder_ids.csv', index=False)